# Project Shark Attacks

link: http://www.sharkattackfile.net/incidentlog.htm

In [304]:
import pandas as pd
import numpy as np
import re

In [305]:
!pwd

/Users/aldreybaldovi/Desktop/ironhack/labs/module_1/week_8/project_shark_attacks


In [306]:
df = pd.read_excel('GSAF5.xls', encoding='latin1')
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2020.09.20,20-Sep-2020,2020.0,Unprovoked,USA,Florida,"Sombero Key Light, Monroe County",Snorkeling,Andrew Charles Eddy,M,...,Bull shark,"Click Orlando, 9/20/2020",2020.09.20-Eddy.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.09.20,2020.09.20,6576.0,NaN,NaN
1,2020.09.17,17-Sep-2020,2020.0,Provoked,AUSTRALIA,Queensland,Fraser Island,Fishing,male,M,...,"""whitetip shark""","K. McMurray, Tracking Sharks.com and B. Myatt,...",2020.09.17-FraserIsland.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.09.17,2020.09.17,6575.0,NaN,NaN
2,2020.09.16.b,16-Sep-2020,2020.0,Unprovoked,AUSTRALIA,New South Wales,Cabarita Beach,Foil-boarding,Christian Bungate,M,...,White shark,"B. Myatt, GSAF",2020.09.16.b-Bungate.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.09.16.b,2020.09.16.b,6574.0,NaN,NaN
3,2020.09.16,16-Sep-2020,2020.0,NaN,USA,Florida,"Daytona Beach Shores, Volusia County",Swimming,Eric Bowman,M,...,Shark involvement not confirmed,"K. McMurray, TrackingSharks.com",2020.09.16-Bowman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.09.16,2020.09.16,6573.0,NaN,NaN
4,2020.09.15.b,15-Sep-2020,2020.0,Unprovoked,USA,Florida,"Ponce Inlet, Volusia County",Surfing,Cole Smyth,M,...,NaN,"K. McMurray, TrackingSharks.com",2020.09.15.b-Smyth.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.09.15.b,2020.09.15.b,6572.0,NaN,NaN


In [307]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [308]:
len(df)

25796

In [309]:
df_bkp = df.copy()

- Analisar a quantidade de vazios nas colunas de interesse ('Year', 'Sex', 'Country', 'Fatal (Y/N)')

In [310]:
df_bkp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25796 entries, 0 to 25795
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8775 non-null   object 
 1   Date                    6575 non-null   object 
 2   Year                    6573 non-null   float64
 3   Type                    6566 non-null   object 
 4   Country                 6525 non-null   object 
 5   Area                    6108 non-null   object 
 6   Location                6025 non-null   object 
 7   Activity                6018 non-null   object 
 8   Name                    6360 non-null   object 
 9   Sex                     6003 non-null   object 
 10  Age                     3672 non-null   object 
 11  Injury                  6545 non-null   object 
 12  Fatal (Y/N)             6023 non-null   object 
 13  Time                    3154 non-null   object 
 14  Species                 3622 non-null 

# HYPOTHESIS: 
- There is more death cases with **man than woman** (SEX AND FATAL (Y/N) COLUMNS) **OK**
- There is more death cases with **man than woman** in all centuries **OK**
- Surf is the activity which causes more accident **in the last years** (YEAR AND ACTIVITY COLUMNS)
- Most of the cases were **unprovoked** (TYPE COLUMN)
- **Young people** is more likely to die by shark attacks (AGE COLUMN)
- The **location** with greater number of cases has no legislation or safety measures (COUNTRY AND AREA)
- Season with more cases is summer (DATE COLUMN AND **NEW COLUMN SEASON**)

STEPS

- Use columns: 

`['Date', 'Year', 'Type', 'Country', 'Area', 'Sex ', 'Age','Species ','Activity','Fatal (Y/N)']`


- Drop columns: 

`['Case Number', 'Name', 'Location', 'Injury', 'Time','Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']`


- Rename columns (df = df.rename(columns = {'Date' : 'date'}) OR df.rename({'Date' : 'date'}, axis =1, inplace = True):
    - lower case
    - no spaces


- Rename similar words in lines:
    - **m:** male, M 
    - **f:** female, F 
    - **y:** Yes, Y, y 
    - **n:** No, no, N


- Analise columns with NAN:
    - Replacing NAN to 0 (`df.fillna(0)`) when necessary


# RESULTS

- Filtering Data Frame and renaming columns

In [311]:
# Filtering data frame with useful columns

df = df.loc[:,['Date', 'Year', 'Type', 'Country', 'Area', 'Sex ', 'Age','Species ','Activity','Fatal (Y/N)']]
df.head()

,Date,Year,Type,Country,Area,Sex,Age,Species,Activity,Fatal (Y/N)
0,20-Sep-2020,2020.0,Unprovoked,USA,Florida,M,30,Bull shark,Snorkeling,N
1,17-Sep-2020,2020.0,Provoked,AUSTRALIA,Queensland,M,50s,"""whitetip shark""",Fishing,N
2,16-Sep-2020,2020.0,Unprovoked,AUSTRALIA,New South Wales,M,NaN,White shark,Foil-boarding,N
3,16-Sep-2020,2020.0,NaN,USA,Florida,M,48,Shark involvement not confirmed,Swimming,N
4,15-Sep-2020,2020.0,Unprovoked,USA,Florida,M,15,NaN,Surfing,N


In [312]:
# Renaming columns

df.rename({'Date':'date', 'Year':'year', 'Type':'type', 'Country':'country', 'Area':'area', 
                 'Sex ':'sex', 'Age':'age','Species ':'species','Activity':'activity',
                 'Fatal (Y/N)':'fatal'}, axis =1, inplace = True)
df.head()

,date,year,type,country,area,sex,age,species,activity,fatal
0,20-Sep-2020,2020.0,Unprovoked,USA,Florida,M,30,Bull shark,Snorkeling,N
1,17-Sep-2020,2020.0,Provoked,AUSTRALIA,Queensland,M,50s,"""whitetip shark""",Fishing,N
2,16-Sep-2020,2020.0,Unprovoked,AUSTRALIA,New South Wales,M,NaN,White shark,Foil-boarding,N
3,16-Sep-2020,2020.0,NaN,USA,Florida,M,48,Shark involvement not confirmed,Swimming,N
4,15-Sep-2020,2020.0,Unprovoked,USA,Florida,M,15,NaN,Surfing,N


## There is more death cases with **man than woman** (DATE, YEAR, SEX AND FATAL (Y/N) COLUMNS)


### **Working with `date` and `year`:**

 - Delete null values
    
    - regex to use only the year
    
        `pattern = '\d\d\d\d'`
        
        `print(re.findall(pattern, text))`
        
    - group the number of fatal cases of the same year
    
        `fatal_cases_year = df.groupby('date', as_index = False)`

In [313]:
df['date'].value_counts()

1957                      9
1942                      8
1958                      7
1941                      6
No date                   6
                         ..
24-Jan-1931               1
Reported 02-April 1906    1
Sep-1921                  1
09-Jun-1993               1
17-Jan-2015               1
Name: date, Length: 5685, dtype: int64

In [314]:
df['year'].value_counts()

2015.0    143
2017.0    139
2016.0    131
2011.0    128
2014.0    127
0.0       126
2008.0    122
2013.0    122
2018.0    121
2009.0    120
2012.0    117
2007.0    113
2019.0    111
2005.0    103
2006.0    103
2010.0    101
2000.0     97
1959.0     93
1960.0     93
2001.0     92
2003.0     92
2004.0     92
2002.0     88
1962.0     86
1961.0     78
1995.0     76
2020.0     75
1964.0     66
1999.0     66
1998.0     65
1963.0     61
1996.0     61
1966.0     58
1997.0     57
1992.0     56
1993.0     56
1994.0     56
1988.0     55
1958.0     54
1989.0     53
1956.0     51
1965.0     51
1983.0     50
1975.0     49
1981.0     49
1967.0     48
1968.0     47
1954.0     45
1955.0     43
1950.0     43
1970.0     42
1982.0     42
1957.0     41
1942.0     41
1984.0     41
1991.0     39
1976.0     39
1986.0     39
1990.0     39
1974.0     38
1929.0     38
1985.0     37
1972.0     36
1953.0     36
1980.0     36
1987.0     35
1936.0     32
1935.0     32
1951.0     32
1944.0     31
1949.0     31
1971.0

In [315]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25796 entries, 0 to 25795
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      6575 non-null   object 
 1   year      6573 non-null   float64
 2   type      6566 non-null   object 
 3   country   6525 non-null   object 
 4   area      6108 non-null   object 
 5   sex       6003 non-null   object 
 6   age       3672 non-null   object 
 7   species   3622 non-null   object 
 8   activity  6018 non-null   object 
 9   fatal     6023 non-null   object 
dtypes: float64(1), object(9)
memory usage: 2.0+ MB


In [316]:
null_lines = df['year'].isnull()
df[null_lines]

,date,year,type,country,area,sex,age,species,activity,fatal
444,Reported 08-Jan-2017,NaN,Invalid,AUSTRALIA,Queensland,M,35,Bull shark,Spearfishing,NaN
6351,Reported 19-Aug-1836,NaN,Unprovoked,ENGLAND,Cumberland,M,NaN,NaN,Swimming,Y
6575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
25791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [317]:
len(df[null_lines])

19223

In [318]:
drop_lines = list(df[null_lines].index)
df = df.drop(drop_lines, axis = 0)
len(df)

6573

In [319]:
zero = df.loc[(df['year'] == 0)]
zero

,date,year,type,country,area,sex,age,species,activity,fatal
6449,Ca. 214 B.C.,0.0,Unprovoked,NaN,Ionian Sea,M,NaN,NaN,Ascending from a dive,Y
6450,Ca. 336.B.C..,0.0,Unprovoked,GREECE,Piraeus,M,NaN,NaN,Washing his pig in preparation for a religious...,Y
6451,493 B.C.,0.0,Sea Disaster,GREECE,Off Thessaly,M,NaN,NaN,Shipwrecked Persian Fleet,Y
6452,Ca. 725 B.C.,0.0,Sea Disaster,ITALY,Tyrrhenian Sea,M,NaN,NaN,Shipwreck,Y
6453,After 2013,0.0,Unprovoked,AUSTRALIA,Queensland,M,NaN,Hammerhead shark,Spearfishing,N
6454,Before 1824,0.0,Unprovoked,AUSTRALIA,Queensland,F,NaN,NaN,Swimming,NaN
6455,Before 1939,0.0,Unprovoked,CANADA,NaN,M,NaN,NaN,Fishing,N
6456,1990 or 1991,0.0,Unprovoked,KENYA,Mombasa,M,NaN,NaN,Diving,N
6457,Before 2016,0.0,Unprovoked,KENYA,Mombasa,M,NaN,NaN,Diving,Y
6458,Before Oct-2009,0.0,Unprovoked,PANAMA,Bocas del Toro Province,M,20,NaN,Swimming/,Y


- **When year is equal 0, it is related to data from around 1900**
    - Before Century XIX: <= 1800
    - Century XIX: 1801 to 1900
    - Century XX: 1901 to 2000
    - Century XXI: 2001 to 2100

In [320]:
df['year'] = df['year'].astype('int')

In [321]:
df

,date,year,type,country,area,sex,age,species,activity,fatal
0,20-Sep-2020,2020,Unprovoked,USA,Florida,M,30,Bull shark,Snorkeling,N
1,17-Sep-2020,2020,Provoked,AUSTRALIA,Queensland,M,50s,"""whitetip shark""",Fishing,N
2,16-Sep-2020,2020,Unprovoked,AUSTRALIA,New South Wales,M,NaN,White shark,Foil-boarding,N
3,16-Sep-2020,2020,NaN,USA,Florida,M,48,Shark involvement not confirmed,Swimming,N
4,15-Sep-2020,2020,Unprovoked,USA,Florida,M,15,NaN,Surfing,N
...,...,...,...,...,...,...,...,...,...,...
6570,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,M,NaN,NaN,Diving,Y
6571,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,M,NaN,NaN,Pearl diving,Y
6572,1900-1905,0,Unprovoked,USA,North Carolina,M,NaN,NaN,Swimming,Y
6573,1883-1889,0,Unprovoked,PANAMA,NaN,M,NaN,NaN,NaN,Y


In [322]:
#Test if the column has null value

df['year'].isnull().value_counts()

False    6573
Name: year, dtype: int64

In [323]:
#to update zero values to 1900 in column `year`
df['year'].loc[(df['year'] == 0)] = 1800

/Users/aldreybaldovi/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [324]:
df['year'].value_counts()

2015    143
2017    139
2016    131
2011    128
1800    127
2014    127
2008    122
2013    122
2018    121
2009    120
2012    117
2007    113
2019    111
2005    103
2006    103
2010    101
2000     97
1959     93
1960     93
2004     92
2003     92
2001     92
2002     88
1962     86
1961     78
1995     76
2020     75
1964     66
1999     66
1998     65
1996     61
1963     61
1966     58
1997     57
1994     56
1993     56
1992     56
1988     55
1958     54
1989     53
1956     51
1965     51
1983     50
1981     49
1975     49
1967     48
1968     47
1954     45
1950     43
1955     43
1970     42
1982     42
1942     41
1984     41
1957     41
1990     39
1986     39
1976     39
1991     39
1974     38
1929     38
1985     37
1972     36
1980     36
1953     36
1987     35
1951     32
1936     32
1935     32
1944     31
1949     31
1969     30
1971     30
1937     30
1947     30
1948     29
1952     29
1931     29
1943     28
1941     27
1934     27
1973     27
1932     27
1977

**Try to find the 4 last numbers in date and replace it after:**

In [325]:
'''df['date'] = df['date'].astype('str')'''

"df['date'] = df['date'].astype('str')"

In [326]:
'''text = [i for i in df['date']]
pattern = '\d\d\d\d$'
df['date_2'] = [re.findall(pattern, text)]
df.head()'''

"text = [i for i in df['date']]\npattern = '\\d\\d\\d\\d$'\ndf['date_2'] = [re.findall(pattern, text)]\ndf.head()"

### **Working with `sex`:**

- Fatality cases per year according to sex:

In [327]:
def fix_sex(x):
    '''
    Function to correct data from df['sex']
    '''
    
    pattern_F = '.*[fF].*'
    pattern_M = '.*[mM].*'
    
    if x != x:
        return 'NaN'
    
    if len(re.findall(pattern_F, x)) == 0 and len(re.findall(pattern_M, x)) == 0:
        return 'NaN'

    
    text = re.sub(pattern_F, 'F', x)
    
    text = re.sub(pattern_M, 'M', text)
    
    
    
    return text

In [328]:
#test

print(fix_sex(' M'))
print(fix_sex('M '))
print(fix_sex(' F'))
print(fix_sex('F '))
print(fix_sex('lli'))
print(fix_sex('M x 2'))
print(fix_sex(np.nan))


M
M
F
F
NaN
M
NaN


In [329]:
df['sex'].value_counts()

M        5299
F         695
M           2
N           2
.           1
lli         1
M x 2       1
Name: sex, dtype: int64

In [330]:
df['sex'] = df['sex'].apply(fix_sex)

In [331]:
df['sex'].value_counts()

M      5302
F       695
NaN     576
Name: sex, dtype: int64

### **Working with `fatal`:**

In [332]:
#Correcting data from df['fatal']

df['fatal'].fillna(0)
df['fatal'].loc[(df['fatal'] == ' Y')] = 'Y'
df['fatal'].loc[(df['fatal'] == 'Y ')] = 'Y'
df['fatal'].loc[(df['fatal'] == 'y')] = 'Y'
df['fatal'].loc[(df['fatal'] == 'y ')] = 'Y'
df['fatal'].loc[(df['fatal'] == ' y')] = 'Y'
df['fatal'].loc[(df['fatal'] == 'Y x 2')] = 'Y'
df['fatal'].loc[(df['fatal'] == 'F')] = 'Y'
df['fatal'].loc[(df['fatal'] == ' N')] = 'N'
df['fatal'].loc[(df['fatal'] == 'N ')] = 'N'
df['fatal'].loc[(df['fatal'] == '2017')] = 'NaN'
df['fatal'].loc[(df['fatal'] == 'M')] = 'NaN'
df['fatal'].loc[(df['fatal'] == 'nan')] = 'NaN'
df['fatal'].loc[(df['fatal'] == 'UNKNOWN')] = 'NaN'


'''df_fatal_year['fatal'].rename({' Y':'Y', 'Y ':'Y', 'y':'Y', 'y ':'Y', ' y':'Y', 'Y x 2':'Y', 'F':'Y',
                               ' N':'N', 'N ':'N', '2017':'UNKNOWN', 'M':'UNKNOWN', 'nan':'UNKNOWN'},
                               axis = 0, inplace = True)
df_fatal_year['fatal'].value_counts()
'''

/Users/aldreybaldovi/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


"df_fatal_year['fatal'].rename({' Y':'Y', 'Y ':'Y', 'y':'Y', 'y ':'Y', ' y':'Y', 'Y x 2':'Y', 'F':'Y',\n                               ' N':'N', 'N ':'N', '2017':'UNKNOWN', 'M':'UNKNOWN', 'nan':'UNKNOWN'},\n                               axis = 0, inplace = True)\ndf_fatal_year['fatal'].value_counts()\n"

In [333]:
df['fatal'].value_counts()

N       4537
Y       1412
NaN       72
2017       1
Name: fatal, dtype: int64

In [334]:
df_fatal_year = df.loc[:,['year', 'fatal', 'sex']]
df_fatal_year

,year,fatal,sex
0,2020,N,M
1,2020,N,M
2,2020,N,M
3,2020,N,M
4,2020,N,M
...,...,...,...
6570,1800,Y,M
6571,1800,Y,M
6572,1800,Y,M
6573,1800,Y,M


### **Answering the question:**

In [335]:
# How many fatal cases in general

fatal_cases = df.loc[df['fatal'] == 'Y']
print(fatal_cases.shape)

(1412, 10)


In [336]:
#How many fatal cases with **man**

sex_M = df.loc[(df['fatal'] == 'Y') & (df['sex'] == 'M'),['fatal','sex']]
print(sex_M.shape)

(1195, 2)


In [337]:
#How many fatal cases with **woman**

sex_F = df.loc[(df['fatal'] == 'Y') & (df['sex'] == 'F'),['fatal','sex']]
print(sex_F.shape)

(108, 2)


## There is more death cases with **man than woman** in all centuries (SEX, FATAL, DATE AND YEAR)


### **Separating intervals between centuries:**

In [338]:
df['fatal'] = df['fatal'].astype('str')

In [339]:
df_fatal_year.groupby('year', as_index = False).count()

,year,fatal,sex
0,5,1,1
1,77,1,1
2,500,1,1
3,1543,1,1
4,1554,1,1
5,1555,1,1
6,1580,1,1
7,1595,1,1
8,1617,1,1
9,1637,1,1


In [340]:
#separating intervals between centuries

#Before century XIX
start_date_BXIX = 0
end_date_BXIX = 1800

#Century XIX
start_date_XIX = 1801
end_date_XIX = 1900

#Century XX
start_date_XX = 1901
end_date_XX = 2000

#Century XXI
start_date_XXI = 2001
end_date_XXI = 2020

#masks:
mask_BXIX = (df_fatal_year['year'] > start_date_BXIX) & (df_fatal_year['year'] <= end_date_BXIX)

mask_XIX = (df_fatal_year['year'] > start_date_XIX) & (df_fatal_year['year'] <= end_date_XIX)

mask_XX = (df_fatal_year['year'] > start_date_XX) & (df_fatal_year['year'] <= end_date_XX)

mask_XXI = (df_fatal_year['year'] > start_date_XXI) & (df_fatal_year['year'] <= end_date_XXI)


### **Answering the question:**

#### Century 18

In [341]:
# Before century XIX : Fatal cases
bxix = df_fatal_year.loc[mask_BXIX].groupby('fatal', as_index = False).count()
bxix.loc[:,['fatal', 'year']]

,fatal,year
0,N,72
1,NaN,8
2,Y,85


In [342]:
# Before century XIX : Fatal cases by sex (male)
(df_fatal_year.loc[mask_BXIX].loc[(df['fatal'] == 'Y') & (df['sex'] == 'M'),['fatal','sex']]).shape

(74, 2)

In [343]:
# Before century XIX : Fatal cases by sex (female)
(df_fatal_year.loc[mask_BXIX].loc[(df['fatal'] == 'Y') & (df['sex'] == 'F'),['fatal','sex']]).shape

(5, 2)

#### Century 19

In [344]:
# Century XIX : Fatal cases
xix = df_fatal_year.loc[mask_XIX].groupby('fatal', as_index = False).count()
xix.loc[:,['fatal', 'year']]

,fatal,year
0,N,223
1,NaN,9
2,Y,278


In [345]:
# Century XIX : Fatal cases : Fatal cases by sex (male)
(df_fatal_year.loc[mask_XIX].loc[(df['fatal'] == 'Y') & (df['sex'] == 'M'),['fatal','sex']]).shape

(256, 2)

In [346]:
# Century XIX : Fatal cases : Fatal cases by sex (female)
(df_fatal_year.loc[mask_XIX].loc[(df['fatal'] == 'Y') & (df['sex'] == 'F'),['fatal','sex']]).shape

(9, 2)

#### Century 20

In [347]:
# Century XX : Fatal cases
xx = df_fatal_year.loc[mask_XX].groupby('fatal', as_index = False).count()
xx.loc[:,['fatal', 'year']]

,fatal,year
0,N,2348
1,NaN,52
2,Y,864


In [348]:
# Century XX : Fatal cases : Fatal cases by sex (male)
(df_fatal_year.loc[mask_XX].loc[(df['fatal'] == 'Y') & (df['sex'] == 'M'),['fatal','sex']]).shape

(714, 2)

In [349]:
# Century XX : Fatal cases : Fatal cases by sex (female)
(df_fatal_year.loc[mask_XX].loc[(df['fatal'] == 'Y') & (df['sex'] == 'F'),['fatal','sex']]).shape

(72, 2)

#### Century 21

In [350]:
# Century XXI : Fatal cases
xxi = df_fatal_year.loc[mask_XXI].groupby('fatal', as_index = False).count()
xxi.loc[:,['fatal', 'year']]

,fatal,year
0,2017,1
1,N,1809
2,NaN,3
3,Y,175


In [351]:
# Century XXI : Fatal cases : Fatal cases by sex (male)
(df_fatal_year.loc[mask_XXI].loc[(df['fatal'] == 'Y') & (df['sex'] == 'M'),['fatal','sex']]).shape

(143, 2)

In [352]:
# Century XXI : Fatal cases : Fatal cases by sex (female)
(df_fatal_year.loc[mask_XXI].loc[(df['fatal'] == 'Y') & (df['sex'] == 'F'),['fatal','sex']]).shape

(22, 2)

In [353]:
bxix_y = bxix['year'][bxix['fatal']=='Y']
xix_y = xix['year'][xix['fatal']=='Y']
xx_y = xx['year'][xx['fatal']=='Y']
xxi_y = xxi['sex'][xxi['fatal']=='Y']

print(f'Before Century XIX: {bxix_y}, Century XIX: {xix_y}, Century XX: {xx_y}, Century XXI: {xxi_y}')

Before Century XIX: 2    85
Name: year, dtype: int64, Century XIX: 2    278
Name: year, dtype: int64, Century XX: 2    864
Name: year, dtype: int64, Century XXI: 3    175
Name: sex, dtype: int64


## **Young people** is more likely to die by shark attacks (AGE COLUMN)


### **Working with `age` column**

In [354]:
df['age'].head

<bound method NDFrame.head of 0        30
1       50s
2       NaN
3        48
4        15
       ... 
6570    NaN
6571    NaN
6572    NaN
6573    NaN
6574     15
Name: age, Length: 6573, dtype: object>

In [355]:
df['age'].value_counts()

16                         86
19                         86
17                         82
18                         82
17                         80
15                         77
18                         73
20                         72
20                         72
21                         67
25                         65
22                         65
15                         65
19                         59
14                         58
16                         57
24                         56
21                         55
24                         54
22                         54
13                         52
14                         51
12                         50
26                         49
25                         48
23                         47
29                         47
28                         47
13                         46
30                         43
27                         41
23                         40
35                         39
26        

In [356]:
null_lines_age = df['age'].isnull()
df[null_lines_age].head()

,date,year,type,country,area,sex,age,species,activity,fatal
2,16-Sep-2020,2020,Unprovoked,AUSTRALIA,New South Wales,M,NaN,White shark,Foil-boarding,N
9,02-Sep-2020,2020,Unprovoked,USA,Florida,M,NaN,"Nurse shark, juvenile",NaN,N
10,30-Aug-2020,2020,Watercraft,AUSTRALIA,New South Wales,NaN,NaN,White shark,Fishing / Filming sharks feeding on whale carcass,N
12,28-Aug-2020,2020,Unprovoked,USA,California,M,NaN,White shark,Kayak fishing,N
16,19-Aug-2020,2020,NaN,USA,South Carolina,F,NaN,NaN,Wading,N


In [357]:
len(df[null_lines_age])

2902

In [358]:
drop_lines_age = list(df[null_lines_age].index)

In [359]:
# Dropping null lines

df_age = df.drop(drop_lines_age, axis = 0)
len(df_age)

3671

In [360]:
df_age['age'].value_counts()

19                         86
16                         86
17                         82
18                         82
17                         80
15                         77
18                         73
20                         72
20                         72
21                         67
25                         65
15                         65
22                         65
19                         59
14                         58
16                         57
24                         56
21                         55
24                         54
22                         54
13                         52
14                         51
12                         50
26                         49
25                         48
28                         47
23                         47
29                         47
13                         46
30                         43
27                         41
23                         40
26                         39
27        

In [361]:
df_age['age'] = df_age['age'].astype('str')

In [362]:
def fix_age(x):
    '''
    Function to correct data from df['age']
    
    This function takes the last numbers of the text
    '''
    
    pattern = '\d?\d$'
    
    if x != x:
        return '0'
    
    if len(re.findall(pattern, x)) == 0: 
        return '0'
    
    text = re.findall(pattern, x)[0]
    
   
    
    return text

In [363]:
#test

fix_age('18 e 12 e 03543350s')

'0'

In [364]:
df_age

,date,year,type,country,area,sex,age,species,activity,fatal
0,20-Sep-2020,2020,Unprovoked,USA,Florida,M,30,Bull shark,Snorkeling,N
1,17-Sep-2020,2020,Provoked,AUSTRALIA,Queensland,M,50s,"""whitetip shark""",Fishing,N
3,16-Sep-2020,2020,NaN,USA,Florida,M,48,Shark involvement not confirmed,Swimming,N
4,15-Sep-2020,2020,Unprovoked,USA,Florida,M,15,NaN,Surfing,N
5,15-Sep-2020,2020,Unprovoked,USA,Florida,M,25,NaN,Surfing,N
...,...,...,...,...,...,...,...,...,...,...
6516,1960s,1800,Unprovoked,IRAQ,Basrah,M,16,Bull shark,Swimming in section of river used for washing ...,N
6527,Before 1957,1800,Provoked,CUBA,Havana Province,M,50,NaN,"Shark fishing, knocked overboard",Y
6538,1941-1942,1800,Unprovoked,IRAQ,Basrah,M,13 or 14,Bull shark,Swimming,Y
6549,Early 1930s,1800,Unprovoked,BELIZE,NaN,M,16,12' tiger shark,Standing,Y


In [365]:
# Applying the function fix_age in df_age['age']

df_age['age'] = df_age['age'].apply(fix_age)

In [366]:
df_age['age'].value_counts()

17    162
18    157
19    147
20    146
16    144
15    142
21    122
22    121
25    113
14    111
24    110
13     99
26     93
23     88
30     85
28     85
12     83
29     82
27     81
32     74
35     74
0      72
10     62
40     58
31     56
34     52
38     51
33     49
43     47
36     46
37     46
11     42
42     41
41     41
50     40
9      39
39     39
45     36
52     35
8      32
48     31
46     31
44     30
47     30
49     30
55     26
7      24
51     23
57     20
60     19
58     18
54     18
6      15
53     14
61     14
59     12
56     11
63     10
70     10
69     10
62      9
5       7
68      7
65      7
64      5
3       5
66      4
75      3
77      3
73      3
71      3
67      2
78      2
1       2
74      2
4       2
86      1
81      1
82      1
84      1
72      1
87      1
Name: age, dtype: int64

### **Answering the question**

In [367]:
df_age['age'] = df_age['age'].astype('int')

In [368]:
# # Defining age groups:

# #kids = 0 to 12
# mask_kids = (df_age['age'] > 0) & (df_age['age'] <= 12)

# #teenager = 13 to 17
# mask_teenager = (df_age['age'] > 12) & (df_age['age'] <= 17)

# #young_adult = 18 to 32
# mask_young_adult = (df_age['age'] > 17) & (df_age['age'] <= 32)

# #adult = 33 to 55
# mask_adult = (df_age['age'] > 32) & (df_age['age'] <= 55)

# #older > 55
# mask_older = (df_age['age'] > 55)


In [369]:
# Defining age groups:
df_age['age_group'] = pd.cut(df_age['age'],[0,12,17,32,55,200], labels = ['kids','teenager','young_adult','adult','older'])
df_age.head()

,date,year,type,country,area,sex,age,species,activity,fatal,age_group
0,20-Sep-2020,2020,Unprovoked,USA,Florida,M,30,Bull shark,Snorkeling,N,young_adult
1,17-Sep-2020,2020,Provoked,AUSTRALIA,Queensland,M,0,"""whitetip shark""",Fishing,N,NaN
3,16-Sep-2020,2020,NaN,USA,Florida,M,48,Shark involvement not confirmed,Swimming,N,adult
4,15-Sep-2020,2020,Unprovoked,USA,Florida,M,15,NaN,Surfing,N,teenager
5,15-Sep-2020,2020,Unprovoked,USA,Florida,M,25,NaN,Surfing,N,young_adult


In [370]:
# Before century XIX : Verifying the number of fatal cases by age group
df_age.loc[mask_BXIX,'century'] = 18
df_age.loc[mask_XIX,'century'] = 19
df_age.loc[mask_XX,'century'] = 20
df_age.loc[mask_XXI,'century'] = 21


df_age


,date,year,type,country,area,sex,age,species,activity,fatal,age_group,century
0,20-Sep-2020,2020,Unprovoked,USA,Florida,M,30,Bull shark,Snorkeling,N,young_adult,21.0
1,17-Sep-2020,2020,Provoked,AUSTRALIA,Queensland,M,0,"""whitetip shark""",Fishing,N,NaN,21.0
3,16-Sep-2020,2020,NaN,USA,Florida,M,48,Shark involvement not confirmed,Swimming,N,adult,21.0
4,15-Sep-2020,2020,Unprovoked,USA,Florida,M,15,NaN,Surfing,N,teenager,21.0
5,15-Sep-2020,2020,Unprovoked,USA,Florida,M,25,NaN,Surfing,N,young_adult,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6516,1960s,1800,Unprovoked,IRAQ,Basrah,M,16,Bull shark,Swimming in section of river used for washing ...,N,teenager,18.0
6527,Before 1957,1800,Provoked,CUBA,Havana Province,M,50,NaN,"Shark fishing, knocked overboard",Y,adult,18.0
6538,1941-1942,1800,Unprovoked,IRAQ,Basrah,M,14,Bull shark,Swimming,Y,teenager,18.0
6549,Early 1930s,1800,Unprovoked,BELIZE,NaN,M,16,12' tiger shark,Standing,Y,teenager,18.0


In [371]:
df_grouped = df_age.groupby(['fatal','age_group','century','sex']).agg(count_cases = ('date','count')).reset_index()
#df_grouped.loc[:,['fatal','age_group','century','sex']]

df_grouped

,fatal,age_group,century,sex,count_cases
0,2017,kids,18.0,F,0
1,2017,kids,18.0,M,0
2,2017,kids,18.0,NaN,0
3,2017,kids,19.0,F,0
4,2017,kids,19.0,M,0
5,2017,kids,19.0,NaN,0
6,2017,kids,20.0,F,0
7,2017,kids,20.0,M,0
8,2017,kids,20.0,NaN,0
9,2017,kids,21.0,F,0


In [372]:
import gc
gc.collect()

20

In [373]:
# Verifying the number of fatal cases by age by sex by century
df_grouped = df_grouped.query('fatal == "Y"')
df_grouped.head()

,fatal,age_group,century,sex,count_cases
180,Y,kids,18.0,F,0
181,Y,kids,18.0,M,0
182,Y,kids,18.0,NaN,0
183,Y,kids,19.0,F,0
184,Y,kids,19.0,M,8


#### Century 18

In [374]:
df_18 = df_grouped.query('century == "18"')

In [375]:
df_18.query('sex == "F"')

,fatal,age_group,century,sex,count_cases
180,Y,kids,18.0,F,0
192,Y,teenager,18.0,F,0
204,Y,young_adult,18.0,F,0
216,Y,adult,18.0,F,0
228,Y,older,18.0,F,0


In [376]:
df_18.query('sex == "M"')

,fatal,age_group,century,sex,count_cases
181,Y,kids,18.0,M,0
193,Y,teenager,18.0,M,3
205,Y,young_adult,18.0,M,1
217,Y,adult,18.0,M,1
229,Y,older,18.0,M,0


#### Century 19

In [377]:
df_19 = df_grouped.query('century == "19"')

In [378]:
df_19.query('sex == "F"')

,fatal,age_group,century,sex,count_cases
183,Y,kids,19.0,F,0
195,Y,teenager,19.0,F,0
207,Y,young_adult,19.0,F,0
219,Y,adult,19.0,F,0
231,Y,older,19.0,F,0


In [379]:
df_19.query('sex == "M"')

,fatal,age_group,century,sex,count_cases
184,Y,kids,19.0,M,8
196,Y,teenager,19.0,M,7
208,Y,young_adult,19.0,M,22
220,Y,adult,19.0,M,5
232,Y,older,19.0,M,0


#### Century 20

In [380]:
df_20 = df_grouped.query('century == "20"')

In [381]:
df_20.query('sex == "F"')

,fatal,age_group,century,sex,count_cases
186,Y,kids,20.0,F,4
198,Y,teenager,20.0,F,9
210,Y,young_adult,20.0,F,19
222,Y,adult,20.0,F,10
234,Y,older,20.0,F,4


In [382]:
df_20.query('sex == "M"')

,fatal,age_group,century,sex,count_cases
187,Y,kids,20.0,M,22
199,Y,teenager,20.0,M,82
211,Y,young_adult,20.0,M,184
223,Y,adult,20.0,M,70
235,Y,older,20.0,M,12


#### Century 21

In [383]:
df_21 = df_grouped.query('century == "21"')

In [384]:
df_21.query('sex == "F"')

,fatal,age_group,century,sex,count_cases
189,Y,kids,21.0,F,1
201,Y,teenager,21.0,F,3
213,Y,young_adult,21.0,F,7
225,Y,adult,21.0,F,4
237,Y,older,21.0,F,7


In [385]:
df_21.query('sex == "M"')

,fatal,age_group,century,sex,count_cases
190,Y,kids,21.0,M,0
202,Y,teenager,21.0,M,13
214,Y,young_adult,21.0,M,56
226,Y,adult,21.0,M,46
238,Y,older,21.0,M,14


## Surf is the activity which causes more accident **in the last years** (YEAR AND ACTIVITY COLUMNS)


### **Working with `activity` column**

- Grouping the activities which shows the greatest number of fatalities

In [386]:
df_fatal_act = df.loc[:,['activity', 'fatal']]
df_fatal_act

,activity,fatal
0,Snorkeling,N
1,Fishing,N
2,Foil-boarding,N
3,Swimming,N
4,Surfing,N
...,...,...
6570,Diving,Y
6571,Pearl diving,Y
6572,Swimming,Y
6573,NaN,Y


In [387]:
df_fatal_act.groupby('activity', as_index = False).count()

,activity,fatal
0,,1
1,,1
2,Diving for abalone (Hookah),1
3,a canoe was pursuing a schooner that had forc...,1
4,"""Boat accident""",1
...,...,...
1547,ship torpedoed 400 miles off the African coas....,1
1548,small boat,1
1549,wreck of the State Oil Company ship Permina,1
1550,yachting accident,1


In [388]:
# pattern = '.*diving.*'
# text = 'I was snorkling and I was bitten'

# re.sub(pattern, 'diving', text)

# pattern = '.*snorkling.*'
# text = 'I was diving and I was bitten'

# re.sub(pattern, 'snorkling', text)

# list_of_activities = ['diving','snorkling']

# for each_activity in list_of_activities:
#     pattern = f'.*{each_activity}.*'
#     text = re.sub(pattern, each_activity, text)
    
# text



In [429]:
def fix_activity(text):
    '''
    Function to correct data from df['activity']
    
    This function takes the text and search for key words 
    (diving, surfing, fishing, snorkeling, swimming, boarding, boat, yacht)
    '''
    if text != text:
        return 'NaN'
    
    text = text.lower()
    
    list_of_activities = ['div', 'surf', 'fish', 'snork', 'swim', 'board', 'boat', 'yacht', 'bath', 'kayak', 'float', 'walk','play']  
    #pattern = '.*(diving).*|.*(surfing).*|.*(fishing).*|.*(snorkling).*|.*(swimming).*|.*(boarding).*|.*(boat).*|.*(yacht).*'
    found = 0

    for each_activity in list_of_activities:
        pattern = f'.*{each_activity}.*'
        text = re.sub(pattern, each_activity, text)
        
        if len(re.findall(pattern, text)) != 0: 
            found += 1
        
    if found == 0:
        text = 'outros'
        

    #[x for x in re.findall(pattern, text)[0] if x != '']

    #re.sub(pattern, [x for x in re.findall(pattern, text)[0] if x != ''][0], text)
   
    
    return text

In [430]:
# test

fix_activity('swimming')

'swim'

In [431]:
df_age['activity_group'] = df_age['activity'].apply(fix_activity)
df_age['activity_group']

0        snork
1         fish
3         swim
4         surf
5         surf
         ...  
6516      swim
6527      fish
6538      swim
6549    outros
6574      swim
Name: activity_group, Length: 3671, dtype: object

In [432]:
pd.options.display.max_rows = 400

In [433]:
df_age['activity_group'].value_counts()

surf      1033
swim       724
fish       531
outros     472
div        325
board      166
NaN        148
snork       88
bath        69
float       37
kayak       24
play        20
walk        18
boat        13
yacht        3
Name: activity_group, dtype: int64

In [434]:
df_grouped_2 = df_age.groupby(['fatal','age_group','century','sex','activity_group']).agg(count_cases = ('date','count')).reset_index()
#df_grouped.loc[:,['fatal','age_group','century','sex']]

df_grouped_2

,fatal,age_group,century,sex,activity_group,count_cases
0,2017,kids,18.0,F,NaN,0
1,2017,kids,18.0,F,bath,0
2,2017,kids,18.0,F,board,0
3,2017,kids,18.0,F,boat,0
4,2017,kids,18.0,F,div,0
...,...,...,...,...,...,...
4495,nan,older,21.0,NaN,snork,0
4496,nan,older,21.0,NaN,surf,0
4497,nan,older,21.0,NaN,swim,1
4498,nan,older,21.0,NaN,walk,0


#### Century 18

In [441]:
df_18_2 = df_grouped_2.query('century == "18"')

In [442]:
df_18_2.query('sex == "M" and count_cases > 0')

,fatal,age_group,century,sex,activity_group,count_cases
927,N,kids,18.0,M,swim,1
1107,N,teenager,18.0,M,swim,2
1279,N,young_adult,18.0,M,div,1
1280,N,young_adult,18.0,M,fish,2
1287,N,young_adult,18.0,M,swim,1
1459,N,adult,18.0,M,div,1
2903,Y,teenager,18.0,M,outros,1
2907,Y,teenager,18.0,M,swim,2
3087,Y,young_adult,18.0,M,swim,1
3260,Y,adult,18.0,M,fish,1


In [443]:
df_18_2.query('sex == "F" and count_cases > 0')

,fatal,age_group,century,sex,activity_group,count_cases
1080,N,teenager,18.0,F,NaN,1
1268,N,young_adult,18.0,F,outros,1


#### Century 19


In [447]:
df_grouped_2.query('century == "19" and sex == "M" and count_cases > 0').sort_values(by = 'activity_group')

,fatal,age_group,century,sex,activity_group,count_cases
3120,Y,young_adult,19.0,M,NaN,1
1140,N,teenager,19.0,M,NaN,1
1320,N,young_adult,19.0,M,NaN,2
2941,Y,teenager,19.0,M,bath,2
4021,nan,young_adult,19.0,M,bath,1
2761,Y,kids,19.0,M,bath,4
3301,Y,adult,19.0,M,bath,1
1501,N,adult,19.0,M,bath,4
3121,Y,young_adult,19.0,M,bath,4
961,N,kids,19.0,M,bath,1


In [448]:
df_grouped_2.query('century == "19" and sex == "F" and count_cases > 0').sort_values(by = 'activity_group')

,fatal,age_group,century,sex,activity_group,count_cases
3645,nan,kids,19.0,F,NaN,1
1126,N,teenager,19.0,F,bath,1
1666,N,older,19.0,F,bath,1
3826,nan,teenager,19.0,F,bath,1


#### Century 20

In [449]:
df_grouped_2.query('century == "20" and sex == "M" and count_cases > 0').sort_values(by = 'activity_group')

,fatal,age_group,century,sex,activity_group,count_cases
1005,N,kids,20.0,M,NaN,6
4245,nan,adult,20.0,M,NaN,2
4065,nan,young_adult,20.0,M,NaN,3
3885,nan,teenager,20.0,M,NaN,4
3705,nan,kids,20.0,M,NaN,3
3525,Y,older,20.0,M,NaN,2
3345,Y,adult,20.0,M,NaN,3
3165,Y,young_adult,20.0,M,NaN,10
2085,NaN,teenager,20.0,M,NaN,2
1725,N,older,20.0,M,NaN,2


In [451]:
df_grouped_2.query('century == "20" and sex == "F" and count_cases > 0').sort_values(by = 'activity_group')

,fatal,age_group,century,sex,activity_group,count_cases
990,N,kids,20.0,F,NaN,1
2970,Y,teenager,20.0,F,NaN,1
3150,Y,young_adult,20.0,F,NaN,1
1530,N,adult,20.0,F,NaN,1
4230,nan,adult,20.0,F,NaN,1
1350,N,young_adult,20.0,F,NaN,1
4050,nan,young_adult,20.0,F,NaN,4
3331,Y,adult,20.0,F,bath,1
2791,Y,kids,20.0,F,bath,1
1351,N,young_adult,20.0,F,bath,2


#### Century 21

In [453]:
df_grouped_2.query('century == "21" and sex == "M" and count_cases > 0').sort_values(by = 'activity_group')

,fatal,age_group,century,sex,activity_group,count_cases
4470,nan,older,21.0,M,NaN,1
4290,nan,adult,21.0,M,NaN,1
1410,N,young_adult,21.0,M,NaN,8
1230,N,teenager,21.0,M,NaN,1
3030,Y,teenager,21.0,M,NaN,1
3210,Y,young_adult,21.0,M,NaN,1
3930,nan,teenager,21.0,M,NaN,2
3390,Y,adult,21.0,M,NaN,3
4110,nan,young_adult,21.0,M,NaN,3
1590,N,adult,21.0,M,NaN,3


In [454]:
df_grouped_2.query('century == "21" and sex == "F" and count_cases > 0').sort_values(by = 'activity_group')

,fatal,age_group,century,sex,activity_group,count_cases
1035,N,kids,21.0,F,NaN,3
1575,N,adult,21.0,F,NaN,2
3735,nan,kids,21.0,F,NaN,1
1395,N,young_adult,21.0,F,NaN,1
1215,N,teenager,21.0,F,NaN,1
1576,N,adult,21.0,F,bath,1
1577,N,adult,21.0,F,board,3
1217,N,teenager,21.0,F,board,8
1397,N,young_adult,21.0,F,board,4
1037,N,kids,21.0,F,board,13


## Most of the cases were **unprovoked** (TYPE COLUMN)


In [461]:
df

,date,year,type,country,area,sex,age,species,activity,fatal
0,20-Sep-2020,2020,Unprovoked,USA,Florida,M,30,Bull shark,Snorkeling,N
1,17-Sep-2020,2020,Provoked,AUSTRALIA,Queensland,M,50s,"""whitetip shark""",Fishing,N
2,16-Sep-2020,2020,Unprovoked,AUSTRALIA,New South Wales,M,NaN,White shark,Foil-boarding,N
3,16-Sep-2020,2020,NaN,USA,Florida,M,48,Shark involvement not confirmed,Swimming,N
4,15-Sep-2020,2020,Unprovoked,USA,Florida,M,15,NaN,Surfing,N
...,...,...,...,...,...,...,...,...,...,...
6570,Before 1903,1800,Unprovoked,AUSTRALIA,Western Australia,M,NaN,NaN,Diving,Y
6571,Before 1903,1800,Unprovoked,AUSTRALIA,Western Australia,M,NaN,NaN,Pearl diving,Y
6572,1900-1905,1800,Unprovoked,USA,North Carolina,M,NaN,NaN,Swimming,Y
6573,1883-1889,1800,Unprovoked,PANAMA,NaN,M,NaN,NaN,NaN,Y


In [460]:
df['type'].value_counts()

Unprovoked             4804
Provoked                601
Invalid                 551
Watercraft              346
Sea Disaster            240
Questionable             12
Boat                      7
Unconfirmed               1
Under investigation       1
Unverified                1
Name: type, dtype: int64

In [465]:
df_type_F = df.query('sex == "F" and fatal == "Y"').sort_values(by = 'type')
df_type_F

,date,year,type,country,area,sex,age,species,activity,fatal
6046,Reported 08-Dec-1884,1884,Provoked,FRANCE,Côte d'Azur,F,NaN,10' shark,NaN,Y
4309,Dec-1959,1959,Sea Disaster,MALDIVE ISLANDS,400 miles southeast of Sri Lanka,F,3,NaN,17 Maldivians adrift in open boat for 31 days,Y
5636,Reported 30-Nov-1912,1912,Sea Disaster,PHILIPPINES,NaN,F,19,NaN,Swimming from foundered vessel,Y
3477,30-Jul-1975,1975,Sea Disaster,USA,Oregon,F,62,NaN,Sea disaster,Y
3335,15-Mar-1980,1980,Sea Disaster,PHILIPPINES,NaN,F,NaN,NaN,Sinking of the ferryboat Bongbong 1,Y
5694,06-Mar-1909,1909,Sea Disaster,SOUTH AFRICA,South Atlantic Ocean,F,NaN,NaN,Wreck of the 1308-ton Norwegian ship Auckland,Y
3188,26-Jul-1983,1983,Sea Disaster,AUSTRALIA,Queensland,F,21,"Tiger shark, 5 m [16.5']",Swimming from the New Venture,Y
5708,13-May-1908,1908,Sea Disaster,INDONESIA,Java,F,NaN,NaN,native boats sunk in storm,Y
26,27-Jul-2020,2020,Unprovoked,USA,Maine,F,63,"White shark, 11'?",Swimming,Y
5173,01-Jul-1935,1935,Unprovoked,CROATIA,Adriatic Sea,F,22,NaN,Swimming,Y


In [466]:
df_type_F['type'].value_counts()

Unprovoked      100
Sea Disaster      7
Provoked          1
Name: type, dtype: int64

In [467]:
df_type_M = df.query('sex == "M" and fatal == "Y"').sort_values(by = 'type')
df_type_M

,date,year,type,country,area,sex,age,species,activity,fatal
3348,05-Dec-1979,1979,Invalid,PORTUGAL,Madeira Islands,M,19,White shark?,Spearfishing,Y
1938,13-Apr-2004,2004,Invalid,TONGA,Nuku'alofa,M,NaN,Questionable Incident,Five men on makeshift raft after their 10 m fi...,Y
6500,"No date, Before 1975",1800,Invalid,PAPUA NEW GUINEA,Milne Bay Province,M,NaN,Questionable,Scuba diving,Y
1349,12-Sep-2009,2009,Invalid,USA,North Carolina,M,60,Shark involvement prior to death not confirmed,Swimming,Y
2159,16-Sep-2001,2001,Invalid,USA,Florida,M,42,Questionable incident - shark bite may have pr...,Wreck / Technical diving,Y
...,...,...,...,...,...,...,...,...,...,...
5135,01-Dec-1936,1936,Watercraft,AUSTRALIA,Victoria,M,50,Thought to involve a 12' white shark,Fishing,Y
2335,Reported 28-Jan-2000,2000,Watercraft,REUNION,NaN,M,NaN,NaN,Canoe with 3 men onboard sank,Y
5357,06-Jan-1929,1929,Watercraft,TURKEY,NaN,M,NaN,NaN,Fishing boat,Y
5137,Reported 11-Sep-1936,1936,NaN,VIETNAM,NaN,M,NaN,NaN,Wreck of a sampam,Y


In [468]:
df_type_M['type'].value_counts()

Unprovoked      1048
Sea Disaster     109
Provoked          16
Invalid            8
Watercraft         8
Questionable       4
Name: type, dtype: int64

## The **location** with greater number of cases has no legislation or safety measures (COUNTRY AND AREA)


In [480]:
# Country with greater number of cases according the activity

df_country = df.groupby(['country', 'activity']).agg(count_cases = ('date','count')).reset_index()
#df_grouped.loc[:,['fatal','age_group','century','sex']]

df_country.sort_values(by = 'count_cases', ascending = False)

,country,activity,count_cases
2049,USA,Surfing,612
2072,USA,Swimming,323
327,AUSTRALIA,Surfing,214
337,AUSTRALIA,Swimming,166
1777,USA,Fishing,125
...,...,...,...
793,INDIA,Washing a dog,1
792,INDIA,Washed off catamaran in the surf,1
791,INDIA,Wading,1
790,INDIA,The Dwarka foundered,1


In [484]:
# Country with greater number of cases

df_country_2 = df.groupby(['country']).agg(count_cases = ('date','count')).reset_index()
#df_grouped.loc[:,['fatal','age_group','century','sex']]

df_country_2.sort_values(by = 'count_cases', ascending = False)

,country,count_cases
206,USA,2367
14,AUSTRALIA,1401
173,SOUTH AFRICA,585
129,NEW ZEALAND,137
147,PAPUA NEW GUINEA,135
16,BAHAMAS,124
23,BRAZIL,114
115,MEXICO,95
91,ITALY,72
62,FIJI,64


In [483]:
df_grouped_3 = df_age.groupby(['fatal','age_group','century','sex','activity_group','country']).agg(count_cases = ('date','count')).reset_index()
#df_grouped.loc[:,['fatal','age_group','century','sex']]

df_grouped_3.sort_values(by = 'count_cases', ascending = False)

,fatal,age_group,century,sex,activity_group,country,count_cases
191964,N,young_adult,21.0,M,surf,USA,145
185889,N,young_adult,20.0,M,surf,USA,116
216264,N,adult,21.0,M,surf,USA,77
161589,N,teenager,20.0,M,surf,USA,60
167664,N,teenager,21.0,M,surf,USA,48
...,...,...,...,...,...,...,...
202734,N,adult,19.0,M,bath,RUSSIA,0
202735,N,adult,19.0,M,bath,SAMOA,0
202736,N,adult,19.0,M,bath,SAUDI ARABIA,0
202737,N,adult,19.0,M,bath,SCOTLAND,0


## Season with more cases is summer (DATE COLUMN AND **NEW COLUMN SEASON**)
